In [71]:

import pandas as pd

In [72]:

df=pd.read_csv('/content/drive/My Drive/data/train_fake.csv')

In [73]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [74]:
df=df.dropna()

In [75]:
X=df.drop('label',axis=1)

In [76]:
y=df['label']

In [77]:
X.shape

(18285, 4)

In [78]:
y.shape

(18285,)

In [79]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [80]:

messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [81]:
messages.reset_index(inplace=True)

In [82]:
import nltk
import re
from nltk.corpus import stopwords

In [83]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [85]:
voc_size = 5000

In [86]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4523, 4189, 4134, 2895, 2739, 3206, 438, 4832, 4221, 1598],
 [1312, 3662, 4568, 2329, 4049, 1655, 2378],
 [1330, 1994, 1741, 4597],
 [1526, 4329, 2023, 3955, 3610, 2059],
 [1514, 4049, 589, 4065, 2416, 3964, 4049, 1758, 1406, 888],
 [3938,
  4210,
  2751,
  3229,
  1184,
  4626,
  3963,
  2055,
  362,
  3778,
  1894,
  4861,
  831,
  1667,
  2378],
 [2119, 2614, 1728, 2692, 4243, 2346, 279, 1858, 1659, 1672, 3694],
 [1737, 2767, 532, 942, 1259, 4581, 4626, 1468, 1659, 1672, 3694],
 [3887, 214, 220, 407, 3390, 2611, 1380, 3216, 4626, 2247],
 [4048, 3729, 2949, 530, 4294, 4056, 4829, 4017],
 [3137, 4460, 3861, 2870, 614, 2613, 2361, 1407, 2818, 2419, 3026],
 [3955, 1439, 2739, 2611, 4626, 1259],
 [1602, 4337, 2615, 1801, 3918, 3844, 961, 2201, 4343],
 [1927, 2159, 3547, 3418, 2922, 578, 2822, 1659, 1672, 3694],
 [846, 530, 1343, 4643, 408, 1659, 1672, 3694],
 [1059, 1318, 1860, 3728, 2864, 2560, 2545, 904, 3199, 3205],
 [4241, 4876, 3662],
 [367, 3593, 2502, 4641, 4626, 141, 1844, 2378

In [87]:

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4832 4221 1598]
 [   0    0    0 ... 4049 1655 2378]
 [   0    0    0 ... 1994 1741 4597]
 ...
 [   0    0    0 ... 1659 1672 3694]
 [   0    0    0 ... 1735 1894 2481]
 [   0    0    0 ... 3555 1171  974]]


In [88]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4523,
       4189, 4134, 2895, 2739, 3206,  438, 4832, 4221, 1598], dtype=int32)

In [89]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [90]:

len(embedded_docs),y.shape

(18285, (18285,))

In [91]:

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [92]:

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [94]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 36ms/step - loss: 0.3370 - accuracy: 0.8385 - val_loss: 0.1948 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1366 - accuracy: 0.9467 - val_loss: 0.2032 - val_accuracy: 0.9220
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0942 - accuracy: 0.9659 - val_loss: 0.2153 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0620 - accuracy: 0.9775 - val_loss: 0.2738 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0390 - accuracy: 0.9877 - val_loss: 0.3456 - val_accuracy: 0.9095
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0245 - accuracy: 0.9925 - val_loss: 0.3786 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0136 - accuracy: 0.9955 - val_loss: 0.5068 - val_accuracy: 0.9123

In [98]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#model.add(Dropout(0.3))
model.add(LSTM(100))
#model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [99]:

y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2140, 1279],
       [ 646, 1970]])

In [100]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6810273405136703